
**In this notebook, we will use data from calcium imaging in CA1 of freely-behaving mice (Lee et al., 2025) to study remapping in the hippocampus.** Original dataset can be found on Zenodo: https://zenodo.org/records/14867736

*You are asked to*

1) Plot calcium imaging and behavioral data in intuitive and informative graphic visualizations

2) Measure basic statistics of animal behavior, and plot the results in easily interpreted graphs

3) Generate event rate maps from combined calcium imaging and behavioral data, and plot statistics such as calcium event rate and spatial reliability for each recorded neuron

4) Decode animal position from calcium imaging data, and plot the results in an intuitive and informative way

Start by loading subset of data from Lee et al. (2025) complete dataset is available on Zenodo

In [ ]:
#@title **Import necessary libraries for data loading and analysis and visualization**
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from scipy import stats
import os, warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib
from sklearn.model_selection import KFold
from scipy.ndimage import gaussian_filter1d, gaussian_filter
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import pearsonr
from glob import glob
import cv2
import joblib
from copy import deepcopy
from sklearn.naive_bayes import GaussianNB
!pip install mat73
from mat73 import loadmat

In [ ]:
#@title **Download data from Dropbox using wget and check file type**
!wget -O data.zip "https://www.dropbox.com/scl/fo/jxwjsrjuqh3at4m6dbcy6/AE1bpRkPJ43U4zTK_tiH3fk?rlkey=gkmaw6307cl3sofkbt0cvetzh&st=di37vg9a&dl=0"

import zipfile
import os

file_path = './data.zip'
if os.path.exists(file_path):
    if zipfile.is_zipfile(file_path):
        print("Downloaded file is a valid zip file. Proceeding to unzip.")
        !unzip -o ./data.zip # -o flag overwrites existing files without prompting
    else:
        print("Downloaded file is NOT a valid zip file. Please check the download link.")
else:
    print("Download failed. The file 'data.zip' was not created.")

--2025-09-08 21:02:24--  https://www.dropbox.com/scl/fo/jxwjsrjuqh3at4m6dbcy6/AE1bpRkPJ43U4zTK_tiH3fk?rlkey=gkmaw6307cl3sofkbt0cvetzh&st=di37vg9a&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc66f14133066d003d2046c7e90b.dl.dropboxusercontent.com/zip_download_get/CUY0UYQUPQrlu0non4jUp2U3Ca_d9mMwaorDoFKWtlmT0Bx3wXpvD-sHFFr79B84H4H72DUfWpJDPJJ_i76mgQGNeHREB4jBtLO7bm1ZtR85xA# [following]
--2025-09-08 21:02:25--  https://uc66f14133066d003d2046c7e90b.dl.dropboxusercontent.com/zip_download_get/CUY0UYQUPQrlu0non4jUp2U3Ca_d9mMwaorDoFKWtlmT0Bx3wXpvD-sHFFr79B84H4H72DUfWpJDPJJ_i76mgQGNeHREB4jBtLO7bm1ZtR85xA
Resolving uc66f14133066d003d2046c7e90b.dl.dropboxusercontent.com (uc66f14133066d003d2046c7e90b.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6031:15::a27d:510f
Connecting to uc66f14133066d00

In [ ]:
main_path = "/content"
dataset = joblib.load(os.path.join(main_path, "minidata"))

In [ ]:
# The dataset is contained in a dictionary with several keys
dataset.keys()

dict_keys(['animal', 'traces', 'envs', 'position'])

###**Dataset keys**
#### animal: animal ID from experiment as a string (QL-CA1-50)
#### traces: calcium traces for 11 days of geometries (sequence 2, days 11 - 21) which has shape [days, neurons, frames]
#### envs: list of environment names as strings
#### position: x- and y-position of animal centre of mass on each day which has shape [days, positions, frames]

#### ***Task 1*** - Plot calcium imaging and behavioral data in intuitive and informative graphic visualizations


####***Task 2*** - Measure basic statistics of animal behavior, and plot the results in easily interpreted graphs

#### ***Task 3*** - Generate event rate maps from combined calcium imaging and behavioral data, and plot statistics such as calcium event rate and spatial reliability for each recorded neuron

#### ***Task 4*** - Decode animal position from calcium imaging data, and plot the results in an intuitive and informative way